<a href="https://colab.research.google.com/github/mitprojects/R_System/blob/main/Recommendation_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
!pip install implicit

In [94]:
import pandas as pd
import scipy.sparse as sparse
import os
import numpy as np
import random
import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [95]:
Data_dir = os.getcwd() + "/drive/MyDrive/Colab Notebooks/"

In [96]:
new_data = pd.read_csv(Data_dir+'new Data/archive/new_RS_data.csv')
new_data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [97]:
new_data.head()

,product_name,content_id,personId,contentId,eventStrength,person_id
0,XPS 13 2-in-1 Laptop,1,-9207251133131336884,-9216926795620865886,3.0,0
1,XPS 17 Laptop,3,-9016528795238256703,-9192549002213406534,1.0,1
2,Inspiron 14 Laptop,7,-9016528795238256703,-9176143510534135851,1.0,1
3,Inspiron 15 3000 Laptop,3,-9016528795238256703,-9192549002213406534,1.0,1
4,Alienware m17 R4 Gaming Laptop,12,-9009798162809551896,-9160910454530522563,1.0,2


In [98]:
sparse_content_person = sparse.csr_matrix((new_data['eventStrength'].astype(float), (new_data['content_id'], new_data['person_id'])))
sparse_person_content = sparse.csr_matrix((new_data['eventStrength'].astype(float), (new_data['person_id'], new_data['content_id'])))

In [99]:
sparse_content_person

<13x128 sparse matrix of type '<class 'numpy.float64'>'
	with 157 stored elements in Compressed Sparse Row format>

In [100]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

In [101]:
alpha = 15
data = (sparse_content_person * alpha).astype('double')

# Fit the model
model.fit(data)

In [102]:
content_id = 5
n_similar = 3

person_vecs = model.user_factors
content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))

scores = content_vecs.dot(content_vecs[content_id]) / content_norms
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[content_id]), key=lambda x: -x[1])

In [103]:
for content in similar:
    idx, score = content
    print(new_data.product_name.loc[new_data.content_id == idx].iloc[0])

Alienware m15 R4 Gaming Laptop
XPS 13 2-in-1 Laptop
New XPS 15 Touch Laptop


In [104]:
def recommend(person_id, sparse_person_content, person_vecs, content_vecs, num_contents=10):
    # Get the interactions scores from the sparse person content matrix
    person_interactions = sparse_person_content[person_id,:].toarray()
    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1
    # Make articles already interacted zero
    person_interactions[person_interactions > 1] = 0
    # Get dot product of person vector and all content vectors
    rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = person_interactions * rec_vector_scaled
    # Sort the indices of the content into order of best recommendations
    content_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    # Start empty list to store titles and scores
    product_name = []
    scores = []

    for idx in content_idx:
        # Append titles and scores to the list
        product_name.append(new_data.product_name.loc[new_data.content_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'product_name': product_name, 'score': scores})

    return recommendations

In [105]:
# Get the trained person and content vectors. We convert them to csr matrices
person_vecs = sparse.csr_matrix(model.user_factors)
content_vecs = sparse.csr_matrix(model.item_factors)



In [107]:
# Create recommendations for person with id 8
person_id = 8

recommendations = recommend(person_id, sparse_person_content, person_vecs, content_vecs)

print(recommendations)

                     product_name     score
0  Alienware m15 R3 Gaming Laptop  0.051205
1  Alienware m17 R3 Gaming Laptop  0.041234
2              Inspiron 17 Laptop  0.038477
3              Inspiron 14 Laptop  0.034614
4  Alienware m15 R4 Gaming Laptop  0.034447
5         Inspiron 14 3000 Laptop  0.033881
6            XPS 13 2-in-1 Laptop  0.033645
7                   XPS 15 Laptop  0.033606
8         New XPS 15 Touch Laptop  0.032792
9  Alienware m17 R4 Gaming Laptop  0.000000
